In [9]:
import requests
import pandas as pd
from datetime import datetime

In [10]:
import requests
import pandas as pd
import time
import os

# ✅ Korrekte CoinGecko-IDs für die gewünschten Kryptowährungen
CRYPTO_LIST = {
    # Top Coins
    "Bitcoin": "bitcoin",
    "Ethereum": "ethereum",
    "Wrapped Ethereum": "weth",
    "Solana": "solana",
    "Avalanche": "avalanche-2",
    "Polkadot": "polkadot",
    "Near Protocol": "near",
    "Polygon": "matic-network",
    "XRP": "ripple",
    "Cardano": "cardano",
    "Cronos": "crypto-com-chain",
    "Vulcan Forged PYR": "vulcan-forged",
    "Chiliz": "chiliz",
    "Illuvium": "illuvium",
    "Ronin": "ronin",
    "Band Protocol": "band-protocol",
    "Optimism": "optimism",
    "Celestia": "celestia",
    "Numerai": "numeraire",
    "Aethir": "aethir",
    "Sui": "sui",
    "Hyperliquid": "hyperliquid",
    "Robinhood Coin": "robinhood",
    "Trump Coin": "trump",
    "USD Coin": "usd-coin",
    "Binance Coin": "binancecoin",
    "Litecoin": "litecoin",
    "Dogecoin": "dogecoin",
    "Tron": "tron",
    "Aave": "aave",
    "Hedera": "hedera-hashgraph",
    "Filecoin": "filecoin",
    "Cosmos": "cosmos",
    "Gala": "gala",
    "The Sandbox": "the-sandbox",
    "Audius": "audius",
    "Render": "render-token",
    "Kusama": "kusama",
    "VeChain": "vechain",
    "Chainlink": "chainlink",
    "Berachain": "berachain",
    "TestCoin": "testcoin",

    # Meme-Coins
    "Shiba Inu": "shiba-inu",
    "Pepe": "pepe",
    "Floki Inu": "floki",
    "Bonk": "bonk",
    "Wojak": "wojak",
    "Mog Coin": "mog-coin",
    "Doge Killer (Leash)": "leash",
    "Baby Doge Coin": "baby-doge-coin",
    "Degen": "degen",
    "Toshi": "toshi",
    "Fartcoin": "fartcoin",
    "Banana": "banana",
    "Kabosu": "kabosu",
    "Husky": "husky",
    "Samoyedcoin": "samoyedcoin",
    "Milkbag": "milkbag"
}


# 🕒 Zeitraum für die letzten 90 Tage
DAYS = 90

# 📌 Lokaler Speicherpfad (Google Drive Sync-Ordner)
DRIVE_PATH = "G:/Meine Ablage/reddit/"
OUTPUT_FILE = os.path.join(DRIVE_PATH, "crypto_prices.csv")

# 📌 Liste zur Speicherung der Kursdaten
all_prices = []

# 📌 API-Abfrage für jede Kryptowährung mit besserem Rate-Limit-Handling
for crypto_name, crypto_id in CRYPTO_LIST.items():
    print(f"📡 Abrufen von Kursdaten für {crypto_name}...")

    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart?vs_currency=usd&days={DAYS}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        prices = data["prices"]  # Liste von [timestamp, price]

        for price_data in prices:
            all_prices.append({
                "date": price_data[0],  # Speichert den Timestamp erstmal als Zahl
                "crypto": crypto_name,
                "price": price_data[1]
            })

        print(f"✅ Erfolgreich abgerufen: {crypto_name}")

    elif response.status_code == 429:
        print(f"⚠️ API-Limit erreicht für {crypto_name}. Warte 60 Sekunden...")
        time.sleep(60)  # ⏳ Wartezeit für Rate-Limit
        continue  # Überspringe und versuche später

    else:
        print(f"⚠️ Fehler beim Abrufen der API für {crypto_name}: {response.status_code}")

    # ⏳ Standardpause von 10 Sekunden zwischen den Requests
    time.sleep(10)

# 📌 DataFrame erstellen
df_prices = pd.DataFrame(all_prices)

# ✅ Datum korrekt formatieren
df_prices["date"] = pd.to_datetime(df_prices["date"], unit="ms").dt.date  # Umwandlung von Timestamp zu Date

# ✅ Daten in Google Drive speichern
df_prices.to_csv(OUTPUT_FILE, sep="|", encoding="utf-8-sig", index=False)

print(f"✅ Kursdaten für die letzten 90 Tage gespeichert: {OUTPUT_FILE}")


📡 Abrufen von Kursdaten für Bitcoin...
✅ Erfolgreich abgerufen: Bitcoin
📡 Abrufen von Kursdaten für Ethereum...
✅ Erfolgreich abgerufen: Ethereum
📡 Abrufen von Kursdaten für Wrapped Ethereum...
✅ Erfolgreich abgerufen: Wrapped Ethereum
📡 Abrufen von Kursdaten für Solana...
✅ Erfolgreich abgerufen: Solana
📡 Abrufen von Kursdaten für Avalanche...
✅ Erfolgreich abgerufen: Avalanche
📡 Abrufen von Kursdaten für Polkadot...
✅ Erfolgreich abgerufen: Polkadot
📡 Abrufen von Kursdaten für Near Protocol...
✅ Erfolgreich abgerufen: Near Protocol
📡 Abrufen von Kursdaten für Polygon...
✅ Erfolgreich abgerufen: Polygon
📡 Abrufen von Kursdaten für XRP...
✅ Erfolgreich abgerufen: XRP
📡 Abrufen von Kursdaten für Cardano...
⚠️ API-Limit erreicht für Cardano. Warte 60 Sekunden...
📡 Abrufen von Kursdaten für Cronos...
✅ Erfolgreich abgerufen: Cronos
📡 Abrufen von Kursdaten für Vulcan Forged PYR...
✅ Erfolgreich abgerufen: Vulcan Forged PYR
📡 Abrufen von Kursdaten für Chiliz...
✅ Erfolgreich abgerufen: Chil

In [11]:
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103354 entries, 0 to 103353
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    103354 non-null  object 
 1   crypto  103354 non-null  object 
 2   price   103354 non-null  float64
dtypes: float64(1), object(2)
memory usage: 2.4+ MB


In [12]:
df_prices.crypto.value_counts()

crypto
Binance Coin           2159
The Sandbox            2159
Celestia               2159
Optimism               2159
Illuvium               2159
Hedera                 2159
Bitcoin                2158
Shiba Inu              2158
Cosmos                 2158
Audius                 2158
Render                 2158
Kusama                 2158
VeChain                2158
Chainlink              2158
Floki Inu              2158
Bonk                   2158
Wojak                  2158
Mog Coin               2158
Doge Killer (Leash)    2158
Baby Doge Coin         2158
Toshi                  2158
Fartcoin               2158
Banana                 2158
Kabosu                 2158
Samoyedcoin            2158
Filecoin               2158
Dogecoin               2158
Aave                   2158
Tron                   2158
Wrapped Ethereum       2158
Solana                 2158
Avalanche              2158
Polkadot               2158
Near Protocol          2158
Polygon                2158
XRP          